In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plot_util
import util
from scipy.stats import entropy

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
kate_path = os.path.join("data/kate_test", "before_cardio_events.csv")
eti_path = os.path.join("data/eti", "Eti_events.csv")

paths = {"young": kate_path, "elderly": eti_path}

In [ ]:
wanted_game_ids = [[0], [1]]

In [ ]:
data_dict = util.extract_data(paths, wanted_game_ids)

In [ ]:
def filter_moves(df: pd.DataFrame):
    return df.query("(not (Details.str.contains('UP'))) and (distance_x != 0)")

In [ ]:
feats, weights = plot_util.prepare_features(
    {
        "young": {"actions": filter_moves(data_dict["young"]["actions"])},
        "elderly": {"actions": filter_moves(data_dict["elderly"]["actions"])},
    },
    [["actions", "velocity_x"], ["actions", "velocity_y"]],
)

Plotting without the fliers

In [ ]:
plot_util.plot_compare_box(feats, ["young", "elderly"], ["px/sec", "px/sec"])

In [ ]:
trajectories_dict, polyms_dict, times_dict = util.collect_trajectories(data_dict, 3)

In [ ]:
def compute_trajectories_feats(trajectories_dict: dict, times_dict: dict):
    res = {key: {} for key in trajectories_dict.keys()}
    for key in trajectories_dict.keys():
        trajectories = trajectories_dict[key]
        times = times_dict[key]
        time_delta = list(
            map(lambda time: np.diff(time) / np.timedelta64(1, "s"), times)
        )
        res[key]["Velocity"] = list(map(util.compute_vel_vec, trajectories, time_delta))
        temp = list(map(util.compute_speed_dir_vec, res[key]["Velocity"]))
        res[key]["Speed"] = [t[0] for t in temp]
        res[key]["Direction"] = [t[1] for t in temp]
        res[key]["Duration"] = list(
            map(lambda time: (time[-1] - time[0]) / np.timedelta64(1, "s"), times)
        )
        res[key]["Entropy"] = list(
            map(lambda t: entropy(t.clip(0)).mean(), trajectories)
        )
    return res

In [ ]:
feature_dict = compute_trajectories_feats(trajectories_dict, times_dict)

Plot of the performed swipes

In [ ]:
_, ax = plt.subplots(ncols=len(trajectories_dict), sharex=True, sharey=True)
titles = []

cut_idx = data_dict['young']['fruits'].query("Details.str.contains('Cut')")['index'].values
cut_coor_idx = cut_idx - 1
cut_coors = data_dict['young']['actions'].query("index.isin(@cut_coor_idx)")[['x', 'y']].values

ax[0].scatter(cut_coors[:,0], -cut_coors[:,1], color='black', marker='X')

r = 0
for t in trajectories_dict['young']:
    inter = len(np.intersect1d(t, cut_coors)) // 2
    if inter > 0:
        r += 1

titles.append(f'Young. Rate of hit swipes={r/len(trajectories_dict['young'])}')

cut_idx = data_dict['elderly']['fruits'].query("Details.str.contains('Cut')")['index'].values
cut_coor_idx = cut_idx - 1
cut_coors = data_dict['elderly']['actions'].query("index.isin(@cut_coor_idx)")[['x', 'y']].values

ax[1].scatter(cut_coors[:,0], -cut_coors[:,1], color='black', marker='X')

r = 0
for t in trajectories_dict['elderly']:
    inter = len(np.intersect1d(t, cut_coors)) // 2
    if inter > 0:
        r += 1

titles.append(f'Elderly. Rate of hit swipes={r/len(trajectories_dict['elderly']):.1f}')

plot_util.plot_compare_trajectories(trajectories_dict, num=50, text_flag=False,
                                    titles=titles, exclude={'young': [8], 'elderly': []}, 
                                    ax=ax);

In [ ]:
feats, weights = plot_util.prepare_features(feature_dict, [["Direction"]])

In [ ]:
plot_util.plot_compare_hists(feats, weights, label=["Young", "Elderly"])

In [ ]:
plot_util.plot_compare_box(feats, ["Young", "Elderly"])

In [ ]:
plot_util.plot_compare_hists(
    feats,
    weights,
    nbins=7,
    label=["Young", "Elderly"],
    xlabel=["Pixels (px)", "px/sec", "sec"],
)

In [ ]:
feats, weights = plot_util.prepare_features(data_dict, [["fruits", "Response_time"]])
plot_util.plot_compare_box(feats, ["Young", "Elderly"])

Distribution of scroll's distance. Not accumulated into a single move, so this is per a single 'flick'. 

PD patients are likely to have shorter scrolls, maybe overall longer DOWN to UP time